# Financial Analysis in Python


created by Thomas Wiecki

https://github.com/twiecki/financial-analysis-python-tutorial/blob/master/1.%20Pandas%20Basics.ipynb

What you will learn in this tutorial
------------------------------------

* Timeseries analysis using Pandas
* Using Google Trends to predict market movements
* Build your own trading strategy using Zipline
* Common trading strategies:
   * Momentum trading
   * Mean-reversion

Different stages towards a successful trading strategy
------------------------------------------------------

1. Data analysis / Idea generation
   * Pandas + Matplotlib (Part I)
2. Backtest strategy
   * Pandas can be used but many limitations (Part II)
   * -> Zipline (Part III)
3. Optimize
   * See my previous [PyData talk](http://blog.quantopian.com/zipline_in_the_cloud/)
4. Forward test strategy (paper trading)
   * Trivial on Quantopian (Part IV)
5. Live trading!
   * Coming soon...

# Pandas basics (Data analysis / Idea generation)

In [ ]:
from IPython.core.display import HTML
HTML("<iframe src=http://pandas.pydata.org width=800 height=350></iframe>")

In [ ]:
import datetime
import numpy as np

import pandas as pd
#import pandas.io.data
from pandas_datareader import data, wb
from pandas import Series, DataFrame
pd.__version__

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rc('figure', figsize=(8, 7))
mpl.__version__


Creating/loading time series data
===================

* From Python structures
* From Yahoo finance
* From CSV files

### From Python structures

In [ ]:
labels = ['a', 'b', 'c', 'd', 'e']
s = Series([1, 2, 3, 4, 5], index=labels)
s

In [ ]:
'b' in s

In [ ]:
s['b']

In [ ]:
mapping = s.to_dict()
mapping

In [ ]:
Series(mapping)

### From Yahoo finance

In [ ]:
aapl = data.get_data_yahoo('AAPL', 
                           start=datetime.datetime(2006, 10, 1), 
                           end=datetime.datetime(2012, 1, 1))
aapl.head()

### From CSV files

In [ ]:
aapl.to_csv('../data/aapl_ohlc.csv')
!head ../data/aapl_ohlc.csv

In [ ]:
df = pd.read_csv('../data/aapl_ohlc.csv', index_col='Date', parse_dates=True)
df.head()

In [ ]:
df.index

Series and DataFrame: First steps
=================================

* Indexing
* Slicing
* Creating new columns

In [ ]:
ts = df['Close'][-10:]
ts

A DataFrame is a collection of Series objects. Slicing out a DataFrame column returns a Series.

In [ ]:
type(ts)

In [ ]:
date = ts.index[5]
date

In [ ]:
ts[date]

In [ ]:
ts[5]

We can also select multiple columns.

In [ ]:
df[['Open', 'Close']].head()

New columns can be added on the fly.

In [ ]:
df['diff'] = df.Open - df.Close
df.head()

...and deleted on the fly.

In [ ]:
del df['diff']
df.head()

Common Financial Computations
-----------------------------

* Moving Average
* Returns

In [ ]:
close_px = df['Adj Close']

In [ ]:
pd.rolling_mean?

In [ ]:
mavg = pd.rolling_mean(close_px, 40)
mavg[-10:]

Returns defined as: 
$$ $$
$$r_t = \frac{p_t - p_{t-1}}{p_{t-1}} = \frac{p_t}{p_{t-1}} - 1$$

In [ ]:
rets = close_px / close_px.shift(1) - 1
rets.head()

Or alternatively `.pct_change()`.

In [ ]:
close_px.pct_change().head()

Plotting Basics
---------------

Series and DataFrames have an associated `.plot()` command (uses Matplotlib behind the scenes).

In [ ]:
close_px.plot(label='AAPL')
mavg.plot(label='mavg')
plt.legend()

## Optional: If there is time...

In [ ]:
df = data.get_data_yahoo(['AAPL', 'GE', 'GOOGL', 'IBM', 'KO', 'MSFT', 'PEP'], 
                          start=datetime.datetime(2010, 1, 1), 
                          end=datetime.datetime(2013, 1, 1))['Adj Close']
df.head()

In [ ]:
rets = df.pct_change()

In [ ]:
plt.scatter(rets.PEP, rets.KO)
plt.xlabel('Returns PEP')
plt.ylabel('Returns KO')

In [ ]:
pd.scatter_matrix(rets, diagonal='kde', figsize=(10, 10));

In [ ]:
corr = rets.corr()
corr

In [ ]:
plt.imshow(corr, cmap='hot', interpolation='none')
plt.colorbar()
plt.xticks(range(len(corr)), corr.columns)
plt.yticks(range(len(corr)), corr.columns);

One thing we are often interested in is the relationship of expected returns and the risk we are taking one. Often there is a trade-off between the two.

Here we use `plt.annotate` to put labels on the scatterplot.

In [ ]:
plt.scatter(rets.mean(), rets.std())
plt.xlabel('Expected returns')
plt.ylabel('Risk')
plt.xlim([-.001,.002])
plt.ylim([.005,.020])
for label, x, y in zip(rets.columns, rets.mean(), rets.std()):
    plt.annotate(
        label, 
        xy = (x, y), xytext = (20, -20),
        textcoords = 'offset points', ha = 'right', va = 'bottom',
        bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))

Data alignment and dealing with missing values
==============

* Join, merge, concatenate
* Filling NaNs

Lets pull some more securities from Yahoo finance and build a DataFrame of the adjusted closing prices.

In [ ]:
series_list = []
securities = ['AAPL', 'GOOGL', 'IBM', 'MSFT']
for security in securities:
    s = data.get_data_yahoo(security, start=datetime.datetime(2011, 10, 1), end=datetime.datetime(2013, 1, 1))['Adj Close']
    s.name = security # Rename series to match security name
    series_list.append(s)

`pd.concat` can be used to concatenate multiple `Series` into one `DataFrame`.

In [ ]:
df = pd.concat(series_list, axis=1)
df.head()

In [ ]:
df.ix[0, 'AAPL'] = np.nan
df.ix[1, ['GOOGL', 'IBM']] = np.nan
df.ix[[1, 2, 3], 'MSFT'] = np.nan

df.head()

Pandas has great support for computing with missing values.

In [ ]:
(df.AAPL + df.GOOGL).head()

One common approach to impute missing values in time series is forward filling.

In [ ]:
df.ffill().head()